In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, make_scorer
import optuna
from deeptlf import DeepTFL

In [3]:
# This is a function that loads training and testing data for 10-fold cross validation
def prepare_train_test_dataset(datasets, test_idx):
  test_df = datasets[test_idx]
  train_df = pd.concat([datasets[i] for i in range(len(datasets)) if i != test_idx])

  X_train = train_df.iloc[:, 1:-1].values
  X_test = test_df.iloc[:, 1:-1].values
  y_train = train_df.iloc[:, -1].values
  y_test = test_df.iloc[:, -1].values

  return X_train, X_test, y_train, y_test

In [4]:
import torch
import time

def train_with_base_deeptlf(datasets, key):
  start_time = time.time()
  print("Currently training base DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = DeepTFL()
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [5]:
def create_objective_for_deeptlf(X_train, X_test, y_train, y_test):
  def objective(trial):
      params = {
          "n_est": trial.suggest_int("n_est", 10, 50, step=5),
          "max_depth": trial.suggest_int("max_depth", 3, 10),
          "drop": trial.suggest_float("drop", 0.1, 0.5, step=0.05),
          "hidden_dim": trial.suggest_int("hidden_dim", 64, 512, step=64),
          "n_layers": trial.suggest_int("n_layers", 2, 6),
          "xgb_lr": trial.suggest_float("xgb_lr", 0.01, 0.5, step=0.01),
      }

      model = DeepTFL(
          n_est=params["n_est"],
          max_depth=params["max_depth"],
          drop=params["drop"],
          hidden_dim=params["hidden_dim"],
          n_layers=params["n_layers"],
          xgb_lr=params["xgb_lr"],
          task="class"
      )
      model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

      y_pred = model.predict(X_test)
      mcc = matthews_corrcoef(y_test, y_pred)

      # Return the accuracy
      return mcc

  return objective

In [6]:
def train_with_best_hyperparameters(datasets, key):
  start_time = time.time()
  print("Currently finetuning DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  # Use the first data as the test for the HPO
  X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, 0)

  objective_function = create_objective_for_deeptlf(X_train, X_test, y_train, y_test)

  study = optuna.create_study(direction='maximize')
  study.optimize(objective_function, n_trials=15, gc_after_trial=True)

  best_params = study.best_params
  print("Best hyperparameters:", best_params)

  # Use best hyperparams to conduct a 10-fold cross validation
  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    model = DeepTFL(
        n_est=best_params["n_est"],
        max_depth=best_params["max_depth"],
        drop=best_params["drop"],
        hidden_dim=best_params["hidden_dim"],
        n_layers=best_params["n_layers"],
        xgb_lr=best_params["xgb_lr"],
        task="class"
    )
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [7]:
# Load the validation datasets

datasets = {}

# All

IX_all_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_1.csv')
IX_all_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_2.csv')
IX_all_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_3.csv')
IX_all_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_4.csv')
IX_all_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_5.csv')
IX_all_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_6.csv')
IX_all_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_7.csv')
IX_all_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_8.csv')
IX_all_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_9.csv')
IX_all_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/nopcc/fold_10.csv')
datasets['IX_all_nopcc'] = [IX_all_nopcc_1, IX_all_nopcc_2, IX_all_nopcc_3, IX_all_nopcc_4, IX_all_nopcc_5,
                                       IX_all_nopcc_6, IX_all_nopcc_7, IX_all_nopcc_8, IX_all_nopcc_9, IX_all_nopcc_10]

IX_all_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_1.csv')
IX_all_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_2.csv')
IX_all_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_3.csv')
IX_all_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_4.csv')
IX_all_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_5.csv')
IX_all_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_6.csv')
IX_all_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_7.csv')
IX_all_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_8.csv')
IX_all_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_9.csv')
IX_all_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc95/fold_10.csv')
datasets['IX_all_pcc95'] = [IX_all_pcc95_1, IX_all_pcc95_2, IX_all_pcc95_3, IX_all_pcc95_4, IX_all_pcc95_5,
                                       IX_all_pcc95_6, IX_all_pcc95_7, IX_all_pcc95_8, IX_all_pcc95_9, IX_all_pcc95_10]

IX_all_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_1.csv')
IX_all_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_2.csv')
IX_all_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_3.csv')
IX_all_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_4.csv')
IX_all_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_5.csv')
IX_all_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_6.csv')
IX_all_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_7.csv')
IX_all_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_8.csv')
IX_all_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_9.csv')
IX_all_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/all/cv/pcc75/fold_10.csv')
datasets['IX_all_pcc75'] = [IX_all_pcc75_1, IX_all_pcc75_2, IX_all_pcc75_3, IX_all_pcc75_4, IX_all_pcc75_5,
                                       IX_all_pcc75_6, IX_all_pcc75_7, IX_all_pcc75_8, IX_all_pcc75_9, IX_all_pcc75_10]

# 500

IX_500_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_1.csv')
IX_500_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_2.csv')
IX_500_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_3.csv')
IX_500_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_4.csv')
IX_500_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_5.csv')
IX_500_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_6.csv')
IX_500_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_7.csv')
IX_500_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_8.csv')
IX_500_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_9.csv')
IX_500_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/nopcc/fold_10.csv')
datasets['IX_500_nopcc'] = [IX_500_nopcc_1, IX_500_nopcc_2, IX_500_nopcc_3, IX_500_nopcc_4, IX_500_nopcc_5,
                                       IX_500_nopcc_6, IX_500_nopcc_7, IX_500_nopcc_8, IX_500_nopcc_9, IX_500_nopcc_10]

IX_500_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_1.csv')
IX_500_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_2.csv')
IX_500_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_3.csv')
IX_500_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_4.csv')
IX_500_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_5.csv')
IX_500_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_6.csv')
IX_500_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_7.csv')
IX_500_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_8.csv')
IX_500_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_9.csv')
IX_500_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc95/fold_10.csv')
datasets['IX_500_pcc95'] = [IX_500_pcc95_1, IX_500_pcc95_2, IX_500_pcc95_3, IX_500_pcc95_4, IX_500_pcc95_5,
                                       IX_500_pcc95_6, IX_500_pcc95_7, IX_500_pcc95_8, IX_500_pcc95_9, IX_500_pcc95_10]

IX_500_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_1.csv')
IX_500_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_2.csv')
IX_500_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_3.csv')
IX_500_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_4.csv')
IX_500_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_5.csv')
IX_500_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_6.csv')
IX_500_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_7.csv')
IX_500_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_8.csv')
IX_500_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_9.csv')
IX_500_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/500/cv/pcc75/fold_10.csv')
datasets['IX_500_pcc75'] = [IX_500_pcc75_1, IX_500_pcc75_2, IX_500_pcc75_3, IX_500_pcc75_4, IX_500_pcc75_5,
                                       IX_500_pcc75_6, IX_500_pcc75_7, IX_500_pcc75_8, IX_500_pcc75_9, IX_500_pcc75_10]

# 400

IX_400_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_1.csv')
IX_400_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_2.csv')
IX_400_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_3.csv')
IX_400_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_4.csv')
IX_400_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_5.csv')
IX_400_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_6.csv')
IX_400_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_7.csv')
IX_400_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_8.csv')
IX_400_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_9.csv')
IX_400_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/nopcc/fold_10.csv')
datasets['IX_400_nopcc'] = [IX_400_nopcc_1, IX_400_nopcc_2, IX_400_nopcc_3, IX_400_nopcc_4, IX_400_nopcc_5,
                                       IX_400_nopcc_6, IX_400_nopcc_7, IX_400_nopcc_8, IX_400_nopcc_9, IX_400_nopcc_10]

IX_400_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_1.csv')
IX_400_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_2.csv')
IX_400_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_3.csv')
IX_400_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_4.csv')
IX_400_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_5.csv')
IX_400_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_6.csv')
IX_400_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_7.csv')
IX_400_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_8.csv')
IX_400_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_9.csv')
IX_400_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc95/fold_10.csv')
datasets['IX_400_pcc95'] = [IX_400_pcc95_1, IX_400_pcc95_2, IX_400_pcc95_3, IX_400_pcc95_4, IX_400_pcc95_5,
                                       IX_400_pcc95_6, IX_400_pcc95_7, IX_400_pcc95_8, IX_400_pcc95_9, IX_400_pcc95_10]

IX_400_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_1.csv')
IX_400_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_2.csv')
IX_400_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_3.csv')
IX_400_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_4.csv')
IX_400_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_5.csv')
IX_400_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_6.csv')
IX_400_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_7.csv')
IX_400_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_8.csv')
IX_400_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_9.csv')
IX_400_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/400/cv/pcc75/fold_10.csv')
datasets['IX_400_pcc75'] = [IX_400_pcc75_1, IX_400_pcc75_2, IX_400_pcc75_3, IX_400_pcc75_4, IX_400_pcc75_5,
                                       IX_400_pcc75_6, IX_400_pcc75_7, IX_400_pcc75_8, IX_400_pcc75_9, IX_400_pcc75_10]

# 300

IX_300_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_1.csv')
IX_300_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_2.csv')
IX_300_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_3.csv')
IX_300_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_4.csv')
IX_300_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_5.csv')
IX_300_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_6.csv')
IX_300_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_7.csv')
IX_300_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_8.csv')
IX_300_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_9.csv')
IX_300_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/nopcc/fold_10.csv')
datasets['IX_300_nopcc'] = [IX_300_nopcc_1, IX_300_nopcc_2, IX_300_nopcc_3, IX_300_nopcc_4, IX_300_nopcc_5,
                                       IX_300_nopcc_6, IX_300_nopcc_7, IX_300_nopcc_8, IX_300_nopcc_9, IX_300_nopcc_10]

IX_300_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_1.csv')
IX_300_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_2.csv')
IX_300_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_3.csv')
IX_300_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_4.csv')
IX_300_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_5.csv')
IX_300_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_6.csv')
IX_300_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_7.csv')
IX_300_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_8.csv')
IX_300_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_9.csv')
IX_300_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc95/fold_10.csv')
datasets['IX_300_pcc95'] = [IX_300_pcc95_1, IX_300_pcc95_2, IX_300_pcc95_3, IX_300_pcc95_4, IX_300_pcc95_5,
                                       IX_300_pcc95_6, IX_300_pcc95_7, IX_300_pcc95_8, IX_300_pcc95_9, IX_300_pcc95_10]

IX_300_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_1.csv')
IX_300_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_2.csv')
IX_300_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_3.csv')
IX_300_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_4.csv')
IX_300_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_5.csv')
IX_300_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_6.csv')
IX_300_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_7.csv')
IX_300_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_8.csv')
IX_300_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_9.csv')
IX_300_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/IX/300/cv/pcc75/fold_10.csv')
datasets['IX_300_pcc75'] = [IX_300_pcc75_1, IX_300_pcc75_2, IX_300_pcc75_3, IX_300_pcc75_4, IX_300_pcc75_5,
                                       IX_300_pcc75_6, IX_300_pcc75_7, IX_300_pcc75_8, IX_300_pcc75_9, IX_300_pcc75_10]


In [8]:
def view_results_df(results_df, key):
    print("10-fold cross validation results for results_df with key:", key)
    for i in range(len(results_df)):
        # Format accuracy and MCC to 3 decimal places
        accuracy = f"{results_df['Accuracy'][i]:.3f}"
        mcc = f"{results_df['MCC'][i]:.3f}"
        print(f"Fold {i}, Accuracy: {accuracy}, MCC: {mcc}")


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
IX_all_nopcc_results_df = train_with_base_deeptlf(datasets, 'IX_all_nopcc')

Currently training base DeepTLF model with dataset key: IX_all_nopcc


Epochs:  60%|██████    | 60/100 [00:11<00:07,  5.36it/s]


Early stopping
Fold 1 Accuracy: 0.8551724137931035 MCC: 0.7134761854342644


Epochs:  46%|████▌     | 46/100 [00:08<00:09,  5.41it/s]


Early stopping
Fold 2 Accuracy: 0.8620689655172413 MCC: 0.7253338070208721


...

Early stopping
Fold 8 Accuracy: 0.7361111111111112 MCC: 0.48141065102030917


Epochs:  30%|███       | 30/100 [00:05<00:12,  5.41it/s]


Early stopping
Fold 9 Accuracy: 0.8055555555555556 MCC: 0.6170919847985359


Epochs:  49%|████▉     | 49/100 [00:09<00:09,  5.41it/s]


Early stopping
Fold 10 Accuracy: 0.8263888888888888 MCC: 0.6558849021501785
Elapsed time: 108.63 seconds


In [11]:
print("\n---------------------------------------------------------------------\n")
IX_all_pcc95_results_df = train_with_base_deeptlf(datasets, 'IX_all_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_all_pcc95


Epochs:  24%|██▍       | 24/100 [00:02<00:08,  8.94it/s]


Early stopping
Fold 1 Accuracy: 0.6689655172413793 MCC: 0.3394404285878732


Epochs:  40%|████      | 40/100 [00:03<00:05, 10.30it/s]


Early stopping
Fold 2 Accuracy: 0.8275862068965517 MCC: 0.6568826269159046


...

Early stopping
Fold 8 Accuracy: 0.7916666666666666 MCC: 0.5842356282783902


Epochs:  35%|███▌      | 35/100 [00:03<00:05, 11.13it/s]


Early stopping
Fold 9 Accuracy: 0.8541666666666666 MCC: 0.7089490077940541


Epochs:  32%|███▏      | 32/100 [00:02<00:05, 11.64it/s]

Early stopping
Fold 10 Accuracy: 0.7916666666666666 MCC: 0.5842356282783902
Elapsed time: 47.37 seconds


In [12]:

print("\n---------------------------------------------------------------------\n")
IX_all_pcc75_results_df = train_with_base_deeptlf(datasets, 'IX_all_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_all_pcc75


Epochs:  25%|██▌       | 25/100 [00:01<00:04, 16.56it/s]


Early stopping
Fold 1 Accuracy: 0.696551724137931 MCC: 0.3959505908442164


Epochs:  31%|███       | 31/100 [00:01<00:04, 15.88it/s]


Early stopping
Fold 2 Accuracy: 0.8344827586206897 MCC: 0.6700956757088835


...

Early stopping
Fold 8 Accuracy: 0.7847222222222222 MCC: 0.5699393984226709


Epochs:  34%|███▍      | 34/100 [00:01<00:03, 17.01it/s]


Early stopping
Fold 9 Accuracy: 0.8263888888888888 MCC: 0.6579381567197659


Epochs:  42%|████▏     | 42/100 [00:02<00:03, 16.32it/s]


Early stopping
Fold 10 Accuracy: 0.8541666666666666 MCC: 0.7139328934618736
Elapsed time: 29.75 seconds


In [13]:

print("\n---------------------------------------------------------------------\n")
IX_500_nopcc_results_df = train_with_base_deeptlf(datasets, 'IX_500_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_500_nopcc


Epochs:  30%|███       | 30/100 [00:02<00:05, 12.37it/s]


Early stopping
Fold 1 Accuracy: 0.79 MCC: 0.5801160348116041


Epochs:  30%|███       | 30/100 [00:03<00:07,  9.92it/s]


Early stopping
Fold 2 Accuracy: 0.85 MCC: 0.7069625178409298


...

Early stopping
Fold 8 Accuracy: 0.81 MCC: 0.6201240372124044


Epochs:  28%|██▊       | 28/100 [00:02<00:07,  9.90it/s]


Early stopping
Fold 9 Accuracy: 0.84 MCC: 0.6940220937885672


Epochs:  29%|██▉       | 29/100 [00:02<00:07,  9.99it/s]


Early stopping
Fold 10 Accuracy: 0.8383838383838383 MCC: 0.6771562097595446
Elapsed time: 56.39 seconds


In [14]:

print("\n---------------------------------------------------------------------\n")
IX_500_pcc95_results_df = train_with_base_deeptlf(datasets, 'IX_500_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_500_pcc95


Epochs:  29%|██▉       | 29/100 [00:01<00:04, 17.54it/s]


Early stopping
Fold 1 Accuracy: 0.79 MCC: 0.5801160348116041


Epochs:  33%|███▎      | 33/100 [00:02<00:04, 15.30it/s]


Early stopping
Fold 2 Accuracy: 0.88 MCC: 0.7606087305741639


...

Early stopping
Fold 8 Accuracy: 0.76 MCC: 0.530722777603022


Epochs:  26%|██▌       | 26/100 [00:01<00:04, 16.43it/s]


Early stopping
Fold 9 Accuracy: 0.72 MCC: 0.44320263021395917


Epochs:  27%|██▋       | 27/100 [00:01<00:04, 18.04it/s]

Early stopping
Fold 10 Accuracy: 0.8080808080808081 MCC: 0.6161699166191568
Elapsed time: 31.37 seconds


In [15]:

print("\n---------------------------------------------------------------------\n")
IX_500_pcc75_results_df = train_with_base_deeptlf(datasets, 'IX_500_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_500_pcc75


Epochs:  35%|███▌      | 35/100 [00:01<00:02, 24.92it/s]


Early stopping
Fold 1 Accuracy: 0.85 MCC: 0.7001400420140049


Epochs:  29%|██▉       | 29/100 [00:01<00:02, 24.11it/s]


Early stopping
Fold 2 Accuracy: 0.82 MCC: 0.64


...

Early stopping
Fold 8 Accuracy: 0.78 MCC: 0.5833333333333334


Epochs:  29%|██▉       | 29/100 [00:01<00:02, 24.12it/s]


Early stopping
Fold 9 Accuracy: 0.77 MCC: 0.5409746322980812


Epochs:  30%|███       | 30/100 [00:01<00:03, 23.18it/s]

Early stopping
Fold 10 Accuracy: 0.8080808080808081 MCC: 0.6163265306122448
Elapsed time: 18.59 seconds


In [16]:

print("\n---------------------------------------------------------------------\n")
IX_400_nopcc_results_df = train_with_base_deeptlf(datasets, 'IX_400_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_400_nopcc


Epochs:  30%|███       | 30/100 [00:02<00:06, 10.26it/s]


Early stopping
Fold 1 Accuracy: 0.8125 MCC: 0.634795895148782


Epochs:  33%|███▎      | 33/100 [00:03<00:06, 10.12it/s]


Early stopping
Fold 2 Accuracy: 0.8 MCC: 0.6007514091861107


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.753778361444409


Epochs:  43%|████▎     | 43/100 [00:04<00:05, 10.43it/s]


Early stopping
Fold 9 Accuracy: 0.8875 MCC: 0.7752423010845508


Epochs:  33%|███▎      | 33/100 [00:03<00:06,  9.85it/s]


Early stopping
Fold 10 Accuracy: 0.8481012658227848 MCC: 0.6972419651176118
Elapsed time: 59.35 seconds


In [17]:

print("\n---------------------------------------------------------------------\n")
IX_400_pcc95_results_df = train_with_base_deeptlf(datasets, 'IX_400_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_400_pcc95


Epochs:  28%|██▊       | 28/100 [00:01<00:03, 18.98it/s]


Early stopping
Fold 1 Accuracy: 0.8 MCC: 0.6123724356957946


Epochs:  27%|██▋       | 27/100 [00:01<00:03, 18.89it/s]


Early stopping
Fold 2 Accuracy: 0.8375 MCC: 0.6752110364284797


...

Early stopping
Fold 8 Accuracy: 0.8625 MCC: 0.7252266687565153


Epochs:  31%|███       | 31/100 [00:01<00:03, 20.68it/s]


Early stopping
Fold 9 Accuracy: 0.875 MCC: 0.753778361444409


Epochs:  27%|██▋       | 27/100 [00:01<00:03, 21.21it/s]

Early stopping
Fold 10 Accuracy: 0.7468354430379747 MCC: 0.4954941831288088
Elapsed time: 30.04 seconds


In [18]:

print("\n---------------------------------------------------------------------\n")
IX_400_pcc75_results_df = train_with_base_deeptlf(datasets, 'IX_400_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_400_pcc75


Epochs:  26%|██▌       | 26/100 [00:00<00:02, 26.44it/s]


Early stopping
Fold 1 Accuracy: 0.7875 MCC: 0.5751797717724086


Epochs:  44%|████▍     | 44/100 [00:01<00:02, 26.88it/s]


Early stopping
Fold 2 Accuracy: 0.875 MCC: 0.7509392614826383


...

Early stopping
Fold 8 Accuracy: 0.85 MCC: 0.7008766440504625


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 26.27it/s]


Early stopping
Fold 9 Accuracy: 0.8875 MCC: 0.7752423010845508


Epochs:  27%|██▋       | 27/100 [00:00<00:02, 27.60it/s]

Early stopping
Fold 10 Accuracy: 0.7468354430379747 MCC: 0.4935897435897436
Elapsed time: 17.40 seconds


In [19]:

print("\n---------------------------------------------------------------------\n")
IX_300_nopcc_results_df = train_with_base_deeptlf(datasets, 'IX_300_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_300_nopcc


Epochs:  41%|████      | 41/100 [00:02<00:03, 18.15it/s]


Early stopping
Fold 1 Accuracy: 0.9 MCC: 0.8017837257372731


Epochs:  38%|███▊      | 38/100 [00:02<00:03, 17.06it/s]


Early stopping
Fold 2 Accuracy: 0.8 MCC: 0.6


...

Early stopping
Fold 8 Accuracy: 0.85 MCC: 0.7035264706814485


Epochs:  43%|████▎     | 43/100 [00:02<00:02, 20.68it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8


Epochs:  56%|█████▌    | 56/100 [00:02<00:02, 19.05it/s]


Early stopping
Fold 10 Accuracy: 0.9322033898305084 MCC: 0.86651380732989
Elapsed time: 43.46 seconds


In [20]:

print("\n---------------------------------------------------------------------\n")
IX_300_pcc95_results_df = train_with_base_deeptlf(datasets, 'IX_300_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_300_pcc95


Epochs:  21%|██        | 21/100 [00:00<00:02, 28.15it/s]


Early stopping
Fold 1 Accuracy: 0.5 MCC: 0.0


Epochs:  44%|████▍     | 44/100 [00:01<00:02, 27.09it/s]


Early stopping
Fold 2 Accuracy: 0.8833333333333333 MCC: 0.7670929478598306


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8017837257372731


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 28.38it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8


Epochs:  46%|████▌     | 46/100 [00:01<00:01, 27.28it/s]

Early stopping
Fold 10 Accuracy: 0.9491525423728814 MCC: 0.8988505747126436
Elapsed time: 24.20 seconds


In [21]:

print("\n---------------------------------------------------------------------\n")
IX_300_pcc75_results_df = train_with_base_deeptlf(datasets, 'IX_300_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: IX_300_pcc75


Epochs:  36%|███▌      | 36/100 [00:00<00:01, 38.39it/s]


Early stopping
Fold 1 Accuracy: 0.85 MCC: 0.7003892132633236


Epochs:  36%|███▌      | 36/100 [00:00<00:01, 36.82it/s]


Early stopping
Fold 2 Accuracy: 0.9166666666666666 MCC: 0.8337966824563376


...

Early stopping
Fold 8 Accuracy: 0.9333333333333333 MCC: 0.8685990362153791


Epochs:  35%|███▌      | 35/100 [00:00<00:01, 39.29it/s]


Early stopping
Fold 9 Accuracy: 0.85 MCC: 0.7003892132633236


Epochs:  73%|███████▎  | 73/100 [00:01<00:00, 36.75it/s]

Early stopping
Fold 10 Accuracy: 0.9491525423728814 MCC: 0.8988505747126436
Elapsed time: 16.26 seconds


In [22]:
print("Base model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_pcc95_results_df, 'IX_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_pcc95_results_df, 'IX_500_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_pcc95_results_df, 'IX_400_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_pcc95_results_df, 'IX_300_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_pcc75')
print('\n---------------------------------------------------------------------\n')


Base model:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_nopcc
Fold 0, Accuracy: 0.855, MCC: 0.713
Fold 1, Accuracy: 0.862, MCC: 0.725
Fold 2, Accuracy: 0.897, MCC: 0.795
Fold 3, Accuracy: 0.840, MCC: 0.682
Fold 4, Accuracy: 0.806, MCC: 0.612
Fold 5, Accuracy: 0.875, MCC: 0.755
Fold 6, Accuracy: 0.778, MCC: 0.559
Fold 7, Accuracy: 0.736, MCC: 0.481
Fold 8, Accuracy: 0.806, MCC: 0.617
Fold 9, Accuracy: 0.826, MCC: 0.656

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_pcc95
Fold 0, Accuracy: 0.669, MCC: 0.339
Fold 1, Accuracy: 0.828, MCC: 0.657
Fold 2, Accuracy: 0.903, MCC: 0.808
Fold 3, Accuracy: 0.826, MCC: 0.653
Fold 4, Accuracy: 0.778, MCC: 0.556
Fold 5, Accuracy: 0.806, MCC: 0.617
Fold 6, Accuracy: 0.792, MCC: 0.598
Fold 7, Accuracy: 0.792, MCC: 0.584
Fold 8, Accuracy: 0.854, MCC: 0.709
Fold 9, Accuracy: 

In [23]:
IX_all_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_all_nopcc')


[I 2025-01-04 19:17:05,942] A new study created in memory with name: no-name-71091317-42be-4131-a35d-415878b30d42


Currently finetuning DeepTLF model with dataset key: IX_all_nopcc


Epochs:  34%|███▍      | 34/100 [00:12<00:24,  2.71it/s]

Early stopping



[I 2025-01-04 19:17:23,787] Trial 0 finished with value: 0.6193386107599298 and parameters: {'n_est': 25, 'max_depth': 6, 'drop': 0.30000000000000004, 'hidden_dim': 384, 'n_layers': 3, 'xgb_lr': 0.39}. Best is trial 0 with value: 0.6193386107599298.
Epochs:  52%|█████▏    | 52/100 [00:32<00:29,  1.60it/s]

...

Early stopping


Fold 9 Accuracy: 0.7986111111111112 MCC: 0.6019434199776582


Epochs:  34%|███▍      | 34/100 [00:21<00:42,  1.56it/s]

Early stopping


Fold 10 Accuracy: 0.7986111111111112 MCC: 0.5972798330092391
Elapsed time: 824.07 seconds


In [24]:
IX_all_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_all_pcc95')


[I 2025-01-04 19:30:50,033] A new study created in memory with name: no-name-491d13df-929a-4b70-b072-5a980d723e7f


Currently finetuning DeepTLF model with dataset key: IX_all_pcc95


Epochs:  26%|██▌       | 26/100 [00:08<00:23,  3.21it/s]

Early stopping



[I 2025-01-04 19:31:02,374] Trial 0 finished with value: 0.46532585436614654 and parameters: {'n_est': 40, 'max_depth': 6, 'drop': 0.15000000000000002, 'hidden_dim': 320, 'n_layers': 4, 'xgb_lr': 0.26}. Best is trial 0 with value: 0.46532585436614654.
Epochs:  26%|██▌       | 26/100 [00:08<00:23,  3.12it/s]

...

Early stopping
Fold 8 Accuracy: 0.8263888888888888 MCC: 0.6558849021501785


Epochs:  45%|████▌     | 45/100 [00:02<00:03, 15.77it/s]


Early stopping
Fold 9 Accuracy: 0.8055555555555556 MCC: 0.6111111111111112


Epochs:  51%|█████     | 51/100 [00:03<00:03, 16.13it/s]

Early stopping
Fold 10 Accuracy: 0.8541666666666666 MCC: 0.7089490077940541
Elapsed time: 187.62 seconds


In [25]:
IX_all_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_all_pcc75')


[I 2025-01-04 19:33:57,665] A new study created in memory with name: no-name-4f17fd84-5014-460a-b1cb-5298abf51e63


Currently finetuning DeepTLF model with dataset key: IX_all_pcc75


Epochs:  31%|███       | 31/100 [00:01<00:04, 16.23it/s]
[I 2025-01-04 19:34:00,373] Trial 0 finished with value: 0.6000000108636515 and parameters: {'n_est': 20, 'max_depth': 5, 'drop': 0.30000000000000004, 'hidden_dim': 512, 'n_layers': 2, 'xgb_lr': 0.16}. Best is trial 0 with value: 0.6000000108636515.


Early stopping


Epochs:  20%|██        | 20/100 [00:01<00:04, 16.87it/s]
[I 2025-01-04 19:34:02,276] Trial 1 finished with value: 0.0 and parameters: {'n_est': 15, 'max_depth': 5, 'drop': 0.45000000000000007, 'hidden_dim': 64, 'n_layers': 6, 'xgb_lr': 0.2}. Best is trial 0 with value: 0.6000000108636515.


...

Early stopping
Fold 8 Accuracy: 0.7986111111111112 MCC: 0.6019434199776582


Epochs:  42%|████▏     | 42/100 [00:02<00:03, 15.33it/s]


Early stopping
Fold 9 Accuracy: 0.8472222222222222 MCC: 0.6947125179709105


Epochs:  28%|██▊       | 28/100 [00:01<00:04, 15.91it/s]


Early stopping
Fold 10 Accuracy: 0.8263888888888888 MCC: 0.6533451640455009
Elapsed time: 103.06 seconds


In [26]:
IX_500_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_500_nopcc')


[I 2025-01-04 19:35:40,735] A new study created in memory with name: no-name-49335331-6944-4b57-bb7e-0cfc991610e4


Currently finetuning DeepTLF model with dataset key: IX_500_nopcc


Epochs:  40%|████      | 40/100 [00:02<00:03, 17.51it/s]
[I 2025-01-04 19:35:44,659] Trial 0 finished with value: 0.7001400420140049 and parameters: {'n_est': 10, 'max_depth': 9, 'drop': 0.2, 'hidden_dim': 128, 'n_layers': 5, 'xgb_lr': 0.2}. Best is trial 0 with value: 0.7001400420140049.


Early stopping


Epochs:  32%|███▏      | 32/100 [00:06<00:13,  5.20it/s]
[I 2025-01-04 19:35:54,589] Trial 1 finished with value: 0.7401480444148052 and parameters: {'n_est': 20, 'max_depth': 6, 'drop': 0.2, 'hidden_dim': 448, 'n_layers': 6, 'xgb_lr': 0.43}. Best is trial 1 with value: 0.7401480444148052.


...

Early stopping
Fold 8 Accuracy: 0.84 MCC: 0.68


Epochs:  32%|███▏      | 32/100 [00:06<00:14,  4.58it/s]


Early stopping
Fold 9 Accuracy: 0.83 MCC: 0.6611912172532103


Epochs:  30%|███       | 30/100 [00:05<00:13,  5.29it/s]


Early stopping
Fold 10 Accuracy: 0.8585858585858586 MCC: 0.7176139434748101
Elapsed time: 315.89 seconds


In [27]:
IX_500_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_500_pcc95')


[I 2025-01-04 19:40:56,633] A new study created in memory with name: no-name-27c29074-48f6-45bc-ae97-613564773e46


Currently finetuning DeepTLF model with dataset key: IX_500_pcc95


Epochs:  28%|██▊       | 28/100 [00:06<00:16,  4.38it/s]


Early stopping


[I 2025-01-04 19:41:07,939] Trial 0 finished with value: 0.6633249580710799 and parameters: {'n_est': 50, 'max_depth': 8, 'drop': 0.15000000000000002, 'hidden_dim': 384, 'n_layers': 6, 'xgb_lr': 0.03}. Best is trial 0 with value: 0.6633249580710799.
Epochs:  41%|████      | 41/100 [00:02<00:03, 17.92it/s]
[I 2025-01-04 19:41:11,295] Trial 1 finished with value: 0.6302948515057396 and parameters: {'n_est': 15, 'max_depth': 4, 'drop': 0.5, 'hidden_dim': 384, 'n_layers': 5, 'xgb_lr': 0.36000000000000004}. Best is trial 0 with value: 0.6633249580710799.


...

Early stopping
Fold 8 Accuracy: 0.8 MCC: 0.6004805767690767


Epochs:  29%|██▉       | 29/100 [00:02<00:05, 14.00it/s]


Early stopping
Fold 9 Accuracy: 0.72 MCC: 0.44320263021395917


Epochs:  31%|███       | 31/100 [00:02<00:05, 13.29it/s]


Early stopping
Fold 10 Accuracy: 0.8181818181818182 MCC: 0.6490229342872016
Elapsed time: 172.89 seconds


In [28]:
IX_500_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_500_pcc75')


[I 2025-01-04 19:43:49,529] A new study created in memory with name: no-name-4ba85e58-3512-4cfb-a6c5-55cfc8421225


Currently finetuning DeepTLF model with dataset key: IX_500_pcc75


Epochs:  43%|████▎     | 43/100 [00:02<00:03, 16.60it/s]
[I 2025-01-04 19:43:53,326] Trial 0 finished with value: 0.5453710851915744 and parameters: {'n_est': 40, 'max_depth': 5, 'drop': 0.45000000000000007, 'hidden_dim': 64, 'n_layers': 2, 'xgb_lr': 0.38}. Best is trial 0 with value: 0.5453710851915744.


Early stopping


Epochs:  36%|███▌      | 36/100 [00:01<00:01, 34.03it/s]
[I 2025-01-04 19:43:54,933] Trial 1 finished with value: 0.68 and parameters: {'n_est': 20, 'max_depth': 3, 'drop': 0.2, 'hidden_dim': 64, 'n_layers': 3, 'xgb_lr': 0.34}. Best is trial 1 with value: 0.68.


...

Early stopping
Fold 8 Accuracy: 0.81 MCC: 0.626166801516252


Epochs:  32%|███▏      | 32/100 [00:01<00:03, 17.80it/s]


Early stopping
Fold 9 Accuracy: 0.79 MCC: 0.5810468272831243


Epochs:  33%|███▎      | 33/100 [00:01<00:03, 17.00it/s]

Early stopping
Fold 10 Accuracy: 0.8585858585858586 MCC: 0.717844077810787
Elapsed time: 80.50 seconds


In [29]:
IX_400_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_400_nopcc')


[I 2025-01-04 19:45:10,038] A new study created in memory with name: no-name-9759ab67-05b2-4874-9828-9360f8a0421d


Currently finetuning DeepTLF model with dataset key: IX_400_nopcc


Epochs:  20%|██        | 20/100 [00:05<00:22,  3.58it/s]

Early stopping



[I 2025-01-04 19:45:23,086] Trial 0 finished with value: -0.2966479394838265 and parameters: {'n_est': 30, 'max_depth': 10, 'drop': 0.5, 'hidden_dim': 256, 'n_layers': 6, 'xgb_lr': 0.26}. Best is trial 0 with value: -0.2966479394838265.
Epochs:  34%|███▍      | 34/100 [00:01<00:02, 27.43it/s]
[I 2025-01-04 19:45:25,917] Trial 1 finished with value: 0.5251641394443731 and parameters: {'n_est': 20, 'max_depth': 3, 'drop': 0.30000000000000004, 'hidden_dim': 64, 'n_layers': 2, 'xgb_lr': 0.17}. Best is trial 1 with value: 0.5251641394443731.


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.75


Epochs:  33%|███▎      | 33/100 [00:02<00:04, 16.38it/s]


Early stopping
Fold 9 Accuracy: 0.85 MCC: 0.7008766440504625


Epochs:  40%|████      | 40/100 [00:02<00:03, 19.25it/s]

Early stopping
Fold 10 Accuracy: 0.8607594936708861 MCC: 0.7238292177224794
Elapsed time: 199.62 seconds


In [30]:
IX_400_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_400_pcc95')


[I 2025-01-04 19:48:29,672] A new study created in memory with name: no-name-e027a945-7fe0-4a31-bf8b-7fe3ab7e7d1a


Currently finetuning DeepTLF model with dataset key: IX_400_pcc95


Epochs:  69%|██████▉   | 69/100 [00:12<00:05,  5.34it/s]


Early stopping


[I 2025-01-04 19:48:47,173] Trial 0 finished with value: 0.7752423010845508 and parameters: {'n_est': 50, 'max_depth': 7, 'drop': 0.45000000000000007, 'hidden_dim': 128, 'n_layers': 2, 'xgb_lr': 0.4}. Best is trial 0 with value: 0.7752423010845508.
Epochs:  31%|███       | 31/100 [00:03<00:08,  8.16it/s]
[I 2025-01-04 19:48:53,398] Trial 1 finished with value: 0.3992747047523453 and parameters: {'n_est': 30, 'max_depth': 8, 'drop': 0.5, 'hidden_dim': 320, 'n_layers': 6, 'xgb_lr': 0.08}. Best is trial 0 with value: 0.7752423010845508.


...

Early stopping
Fold 8 Accuracy: 0.85 MCC: 0.7080104323938431


Epochs:  56%|█████▌    | 56/100 [00:10<00:07,  5.60it/s]


Early stopping
Fold 9 Accuracy: 0.95 MCC: 0.9


Epochs:  31%|███       | 31/100 [00:05<00:12,  5.33it/s]


Early stopping
Fold 10 Accuracy: 0.7215189873417721 MCC: 0.4438743696976885
Elapsed time: 246.66 seconds


In [31]:
IX_400_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_400_pcc75')


Currently finetuning DeepTLF model with dataset key: IX_400_pcc75


[I 2025-01-04 19:52:36,346] A new study created in memory with name: no-name-821fd991-2ce0-4376-b6fa-639225a7eda1
Epochs:  37%|███▋      | 37/100 [00:01<00:02, 25.01it/s]
[I 2025-01-04 19:52:38,312] Trial 0 finished with value: 0.356034497458156 and parameters: {'n_est': 20, 'max_depth': 4, 'drop': 0.35, 'hidden_dim': 64, 'n_layers': 6, 'xgb_lr': 0.15000000000000002}. Best is trial 0 with value: 0.356034497458156.


Early stopping


Epochs:  31%|███       | 31/100 [00:01<00:02, 30.94it/s]
[I 2025-01-04 19:52:39,937] Trial 1 finished with value: 0.730731314484506 and parameters: {'n_est': 15, 'max_depth': 10, 'drop': 0.15000000000000002, 'hidden_dim': 256, 'n_layers': 5, 'xgb_lr': 0.15000000000000002}. Best is trial 1 with value: 0.730731314484506.


Early stopping


...

Early stopping
Fold 8 Accuracy: 0.8875 MCC: 0.7752423010845508


Epochs:  36%|███▌      | 36/100 [00:01<00:03, 18.88it/s]


Early stopping
Fold 9 Accuracy: 0.9125 MCC: 0.8252579334125864


Epochs:  26%|██▌       | 26/100 [00:01<00:03, 19.50it/s]


Early stopping
Fold 10 Accuracy: 0.8227848101265823 MCC: 0.6583201802666745
Elapsed time: 65.13 seconds


In [32]:
IX_300_nopcc_results_df = train_with_best_hyperparameters(datasets, 'IX_300_nopcc')


[I 2025-01-04 19:53:41,489] A new study created in memory with name: no-name-2c45e1d7-53b0-47bf-acf9-4ddb32e1e3c5


Currently finetuning DeepTLF model with dataset key: IX_300_nopcc


Epochs:  35%|███▌      | 35/100 [00:04<00:07,  8.43it/s]
[I 2025-01-04 19:53:50,084] Trial 0 finished with value: 0.6681531047810609 and parameters: {'n_est': 35, 'max_depth': 8, 'drop': 0.2, 'hidden_dim': 384, 'n_layers': 3, 'xgb_lr': 0.01}. Best is trial 0 with value: 0.6681531047810609.


Early stopping


Epochs:  36%|███▌      | 36/100 [00:08<00:14,  4.29it/s]


...

Early stopping
Fold 8 Accuracy: 0.8333333333333334 MCC: 0.6681531047810609


Epochs:  30%|███       | 30/100 [00:03<00:08,  8.18it/s]


Early stopping
Fold 9 Accuracy: 0.6833333333333333 MCC: 0.42339019740572553


Epochs:  46%|████▌     | 46/100 [00:06<00:08,  6.73it/s]


Early stopping
Fold 10 Accuracy: 0.9152542372881356 MCC: 0.8310344827586207
Elapsed time: 236.94 seconds


In [33]:
IX_300_pcc95_results_df = train_with_best_hyperparameters(datasets, 'IX_300_pcc95')


[I 2025-01-04 19:57:38,442] A new study created in memory with name: no-name-cf2fe0bc-5b22-4917-9939-4ccd8e11a2f7


Currently finetuning DeepTLF model with dataset key: IX_300_pcc95


Epochs:  52%|█████▏    | 52/100 [00:01<00:01, 27.72it/s]
[I 2025-01-04 19:57:41,546] Trial 0 finished with value: 0.7333333333333333 and parameters: {'n_est': 20, 'max_depth': 5, 'drop': 0.25, 'hidden_dim': 512, 'n_layers': 2, 'xgb_lr': 0.46}. Best is trial 0 with value: 0.7333333333333333.


Early stopping


Epochs:  35%|███▌      | 35/100 [00:03<00:07,  9.04it/s]
[I 2025-01-04 19:57:48,389] Trial 1 finished with value: 0.29417420270727607 and parameters: {'n_est': 40, 'max_depth': 5, 'drop': 0.5, 'hidden_dim': 256, 'n_layers': 6, 'xgb_lr': 0.21000000000000002}. Best is trial 0 with value: 0.7333333333333333.


...

Early stopping
Fold 8 Accuracy: 0.85 MCC: 0.7035264706814485


Epochs:  36%|███▌      | 36/100 [00:01<00:02, 29.62it/s]


Early stopping
Fold 9 Accuracy: 0.85 MCC: 0.7198707287006141


Epochs:  73%|███████▎  | 73/100 [00:02<00:00, 31.14it/s]


Early stopping
Fold 10 Accuracy: 0.9830508474576272 MCC: 0.9666666666666667
Elapsed time: 126.12 seconds


In [34]:
IX_300_pcc75_results_df = train_with_best_hyperparameters(datasets, 'IX_300_pcc75')

[I 2025-01-04 19:59:44,575] A new study created in memory with name: no-name-db200e6a-42bc-47dd-a01e-b66dbec0436e


Currently finetuning DeepTLF model with dataset key: IX_300_pcc75


Epochs:  31%|███       | 31/100 [00:01<00:02, 30.87it/s]
[I 2025-01-04 19:59:46,339] Trial 0 finished with value: 0.6681531047810609 and parameters: {'n_est': 35, 'max_depth': 4, 'drop': 0.2, 'hidden_dim': 384, 'n_layers': 4, 'xgb_lr': 0.24000000000000002}. Best is trial 0 with value: 0.6681531047810609.


Early stopping


Epochs:  40%|████      | 40/100 [00:01<00:01, 33.00it/s]
[I 2025-01-04 19:59:48,207] Trial 1 finished with value: 0.5747048932153912 and parameters: {'n_est': 15, 'max_depth': 5, 'drop': 0.5, 'hidden_dim': 512, 'n_layers': 5, 'xgb_lr': 0.38}. Best is trial 0 with value: 0.6681531047810609.


...

Early stopping
Fold 8 Accuracy: 0.8833333333333333 MCC: 0.7705289916987292


Epochs:  32%|███▏      | 32/100 [00:03<00:07,  8.77it/s]


Early stopping
Fold 9 Accuracy: 0.8666666666666667 MCC: 0.7399400733959437


Epochs:  36%|███▌      | 36/100 [00:04<00:07,  8.46it/s]


Early stopping
Fold 10 Accuracy: 0.864406779661017 MCC: 0.7301081770474639
Elapsed time: 102.92 seconds


In [35]:
print("Finetuned")
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_pcc95_results_df, 'IX_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_all_nopcc_results_df, 'IX_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_pcc95_results_df, 'IX_500_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_500_nopcc_results_df, 'IX_500_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_pcc95_results_df, 'IX_400_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_400_nopcc_results_df, 'IX_400_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_pcc95_results_df, 'IX_300_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(IX_300_nopcc_results_df, 'IX_300_pcc75')
print('\n---------------------------------------------------------------------\n')


Finetuned

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_nopcc
Fold 0, Accuracy: 0.821, MCC: 0.650
Fold 1, Accuracy: 0.855, MCC: 0.710
Fold 2, Accuracy: 0.883, MCC: 0.767
Fold 3, Accuracy: 0.778, MCC: 0.563
Fold 4, Accuracy: 0.854, MCC: 0.712
Fold 5, Accuracy: 0.792, MCC: 0.583
Fold 6, Accuracy: 0.750, MCC: 0.505
Fold 7, Accuracy: 0.806, MCC: 0.613
Fold 8, Accuracy: 0.799, MCC: 0.602
Fold 9, Accuracy: 0.799, MCC: 0.597

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: IX_all_pcc95
Fold 0, Accuracy: 0.772, MCC: 0.545
Fold 1, Accuracy: 0.841, MCC: 0.688
Fold 2, Accuracy: 0.890, MCC: 0.780
Fold 3, Accuracy: 0.819, MCC: 0.651
Fold 4, Accuracy: 0.764, MCC: 0.529
Fold 5, Accuracy: 0.792, MCC: 0.584
Fold 6, Accuracy: 0.792, MCC: 0.589
Fold 7, Accuracy: 0.826, MCC: 0.656
Fold 8, Accuracy: 0.806, MCC: 0.611
Fold 9, Accuracy: 0.